Step-by-step implementation plan
Phase 1: Schema and data readiness
- Add tables: transaction_calculations, partner_calculations.
- Add field: transactions.buyer_is_member.
- Ensure references: partners.user_id → users.id; keep partner list consistent per transaction.
Phase 2: Backend service (Node.js or PHP)
- Create a calculation job/service:
- Input: transaction_id (or run nightly for all “confirmed” transactions without calculations).
- Fetch: transaction + its partners + buyer membership + profit/fee.
- Compute: all fields outlined above.
- Insert: one row into transaction_calculations; one row per partner into partner_calculations.
- Update wallets (optional now, recommended later):
- Seller wallet += seller_profit
- Buyer wallet += buyer_credit (if purchase credit, you can track credit-type in an additional ledger table)
- Partner wallet += partner_credit
- Idempotency: Ensure you don’t duplicate inserts; either upsert (replace) or check if a calculation already exists.
Phase 3: Front-end integration
- Transaction detail page (admin/seller):
- Show calculation snapshot from transaction_calculations (net, distributions).
- Show per-partner table from partner_calculations.
- Profile menu aggregation:
- Seller: total seller_profit, recent transactions, wallet balance.
- Partner: recent partner_credits, deal counts, wallet balance.
- Buyer: buyer_credit history and wallet balance.
- Excel export:
- Allow export of transaction calculations and partner breakdowns per date range.

Data consistency and audit
- Logs: Write a log entry when calculations are generated or updated: user, action (“calculate_transaction”), transaction_id, summary.
- Snapshots: Storing buyer_is_member at transaction time ensures historical accuracy if membership changes later.
- Idempotent processing: Use a unique index on partner_calculations(transaction_id, partner_id) and transaction_calculations(transaction_id).
```
sql

ALTER TABLE partner_calculations
  ADD UNIQUE KEY uniq_tx_partner (transaction_id, partner_id);

ALTER TABLE transaction_calculations
  ADD UNIQUE KEY uniq_tx (transaction_id);

```



Minimal API endpoints to wire this up
- POST /api/transactions/:id/calculate
- Body: none (or override flags)
- Action: run calculation and persist rows
- Returns: transaction_calculations + partner_calculations snapshot
- GET /api/transactions/:id/calculations
- Returns the computed data for display
- GET /api/partners/:id/calculations
- Returns partner’s historical credits
- POST /api/wallets/apply-credits
- Applies pending credits to wallets (if you separate ledger from balance updates)


⚙️ Step 2: Set Up the Backend
- Choose your backend stack (Node.js + Express is my recommendation for you).
- Create models for each table (users, products, transactions, etc.).
- Implement authentication (login with phone + password, hashed).
- Add middleware for role checks (query user_roles to enforce permissions).

📂 Step 3: Core APIs
Build REST endpoints for the main flows:
- Users: register, login, assign roles.
- Products: add, update, import from Excel.
- Transactions: create, confirm, list.
- Calculations: run transaction + partner calculations.
- Wallets: credit/debit, view balance.
- Logs: record every action.

📊 Step 4: Calculation Engine
- Implement the calculation logic we documented:
- Net profit, seller profit, buyer credit, partner shares, dev/team shares.
- Write a service function that:
- Reads a transaction.
- Computes all values.
- Inserts into transaction_calculations and partner_calculations.
- Updates wallets.
- Logs the action.

🖥 Step 5: Frontend Integration
- Add profile pages that aggregate:
- User info (from users + profiles).
- Wallet balance.
- Products owned.
- Transactions and calculations.
- Sellers: Excel upload panel.
- Buyers: transaction history + credits.
- Partners: deal index + share history.

✅ Step 6: Testing
- Unit test each function (especially calculations).
- Test Excel import/export with sample files.
- Test role enforcement (seller can’t edit another seller’s products).
- Test wallet updates are atomic.
